# Segment IDs authorship information


For publication purposes following the FlyWire principles, this notebook is used to retrive following information:
- Number of edits aggregated by lab for each ID
- ID identification, first person and lab that did it
- ID completion status, the person and lab that did it

In [1]:
# Importing packages

import os
import pandas as pd
from fafbseg import flywire
from caveclient import CAVEclient
client = CAVEclient('flywire_fafb_production')
from openpyxl import load_workbook

C:\Users\sebas\anaconda3\envs\EM\lib\site-packages\blessed\terminal.py:186: UserWarning: Failed to setupterm(kind='xterm-color'): Could not find terminal xterm-color
  warnings.warn(msg)


In [20]:
# Loading data set with IDs of interest

# Choose path and file
PC_disc = 'D'
#dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\submission_nature'
dataPath = f'{PC_disc}:\FlyWire-Data\FlyWire-Authorship'
#date = '20230823'
#fileName = f'Updated_unique_seg_ids_in_publication_{date}.xlsx'
fileName = f'Segment IDs in publication_20240115.xlsx'
filePath = os.path.join(dataPath,fileName)

#Loading file as DataFrame
df = pd.read_excel(filePath)


#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["seg_id"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)#
    
# Adding info if missing:
if 'Updated_seg_ids' not in df.columns:
    df["Updated_seg_ids"] = df["seg_id"]
    
#Dropping dupllicates
segment_publication_df = df.drop_duplicates(subset=["seg_id"], keep='first')


#Quick look on the dataframe
display(segment_publication_df.head())
    
#Getting the lists of IDs to update
segmentIDs = df["seg_id"].tolist()


,symbol,hemisphere,seg_id,optic_lobe_id,backbone proofread (Y/N),Updated_seg_ids
0,Tm9,R,720575940632916783,R1,Y,720575940632916783
1,Tm9,R,720575940628906795,R2,Y,720575940628906795
2,Tm9,R,720575940639049038,R3,Y,720575940639049038
3,Tm9,R,720575940641658676,R4,Y,720575940641658676
4,Tm9,R,720575940638132058,R5,Y,720575940638132058


In [3]:

len(segmentIDs)

5188

In [4]:
# Preventing errors before running the hard part
for s in segmentIDs:
    if not type(s) == str:
        print(f'{s} is not a string')

720575940632916783 is not a string
720575940628906795 is not a string
720575940639049038 is not a string
720575940641658676 is not a string
720575940638132058 is not a string
720575940639136600 is not a string
720575940627174415 is not a string
720575940624226795 is not a string
720575940616225209 is not a string
720575940627273487 is not a string
720575940627471887 is not a string
720575940616789734 is not a string
720575940613479074 is not a string
720575940642371981 is not a string
720575940626393858 is not a string
720575940627031570 is not a string
720575940637834867 is not a string
720575940627331599 is not a string
720575940604494694 is not a string
720575940645816099 is not a string
720575940633726364 is not a string
720575940604080318 is not a string
720575940627095340 is not a string
720575940637762879 is not a string
720575940608295726 is not a string
720575940618650166 is not a string
720575940618326875 is not a string
720575940628769040 is not a string
720575940625753744 i

720575940625361552 is not a string
720575940626924184 is not a string
720575940629263003 is not a string
720575940619250334 is not a string
720575940615539359 is not a string
720575940642661024 is not a string
720575940620651174 is not a string
720575940613253799 is not a string
720575940612766378 is not a string
720575940633875117 is not a string
720575940632070829 is not a string
720575940620874416 is not a string
720575940619559600 is not a string
720575940607941298 is not a string
720575940635458228 is not a string
720575940624319157 is not a string
720575940615760569 is not a string
720575940630698690 is not a string
720575940630729410 is not a string
720575940599333574 is not a string
720575940619266763 is not a string
720575940622727885 is not a string
720575940612600529 is not a string
720575940634329811 is not a string
720575940617505492 is not a string
720575940617536212 is not a string
720575940622625494 is not a string
720575940617474777 is not a string
720575940641655515 i

### Getting number of edits and aggregate by lab

In [5]:
# Getting edits
#edits = flywire.fetch_edit_history(unique_segmentIDs[0:500])
#few_segmentIDs = segmentIDs[0:1000]
#edits = flywire.fetch_edit_history(few_segmentIDs)


# Getting edits in a for loop

starting_round = 0 # usually it must starts from 0
_steps = 500
_start = _steps*starting_round
_last = (_steps*starting_round) + _steps
_rounds =  round(len(segmentIDs)/ _steps)

print(f'Rounds to perform: {_rounds}')
edits = pd.DataFrame()
for i in range(starting_round,_rounds):
    print(f'Round {i}')
    # getting edits in batches
    try:
        curr_edits = flywire.fetch_edit_history(segmentIDs[_start:_last])
        edits = pd.concat([edits,curr_edits])
        
    # getting edits from single ids   
    except:
        for i in range(_start,_last):
            curr_id = segmentIDs[i]
            try:
                curr_edits = flywire.fetch_edit_history(curr_id)
                edits = pd.concat([edits,curr_edits])
            except:
                print(f'Faulty id: {curr_id}')
        
    _start = _start + _steps
    _last = _last + _steps
    

Rounds to perform: 10
Round 0


Fetching:   0%|          | 0/500 [00:00<?, ?it/s]

Round 1


Fetching:   0%|          | 0/500 [00:00<?, ?it/s]

Round 2


...

Round 8


Fetching:   0%|          | 0/500 [00:00<?, ?it/s]

Faulty id: 720575940609934019
Round 9


Fetching:   0%|          | 0/500 [00:00<?, ?it/s]

Faulty id: 720575940630880194


In [14]:
edits

,after_root_ids,before_root_ids,is_merge,operation_id,segment,timestamp,user_affiliation,user_id,user_name
0,[720575940622168617],"[720575940608154282, 720575940609856426]",1.0,1136405.0,720575940632916783,2022-03-01 23:36:51.495,Sung Soo Kim Lab,100,Dustin Garner
1,[720575940628493328],"[720575940601227084, 720575940622037160]",1.0,1955389.0,720575940632916783,2022-08-29 13:25:11.666,Alexander Borst Lab,153,Amalia Braun
2,[720575940623305875],"[720575940622168617, 720575940628493328]",1.0,1955390.0,720575940632916783,2022-08-29 13:25:16.836,Alexander Borst Lab,153,Amalia Braun
3,[720575940632916783],"[720575940607142350, 720575940623305875]",1.0,2886901.0,720575940632916783,2023-04-11 08:06:44.112,"Mala Murthy Lab, Sebastian Seung Lab",2762,Nelsie Panes
0,[720575940624254025],"[720575940599998492, 720575940637198733]",1.0,2275648.0,720575940628906795,2022-10-26 00:26:44.114,"Mala Murthy Lab, Sebastian Seung Lab",2754,Cathy Pilapil
...,...,...,...,...,...,...,...,...,...
7,[720575940623499445],"[720575940441760761, 720575940616900793]",1.0,1675720.0,720575940633772708,2022-07-09 20:04:28.962,Eyewire,2815,Nseraf
8,[720575940621369438],"[720575940582302356, 720575940623499445]",1.0,1675721.0,720575940633772708,2022-07-09 20:19:19.471,Eyewire,2815,Nseraf
9,[720575940635430847],"[720575940615158942, 720575940621369438]",1.0,2690063.0,720575940633772708,2023-02-01 03:44:25.975,"Mala Murthy Lab, Sebastian Seung Lab",2766,marchan manaytay
10,[720575940645733156],[720575940635430847],0.0,3133010.0,720575940633772708,2023-09-25 04:15:02.179,"Mala Murthy Lab, Sebastian Seung Lab",2760,Alvin Josh Mandahay


### Current version

In [32]:
# Calculating the total number of entries for each "user_name" within each unique "segment"
user_name_counts = edits.groupby(['segment', 'user_name'])['user_name'].count().reset_index(name='user_edit_count')
# Merging with the original DataFrame to get 'user_affiliation'
user_name_counts = user_name_counts.merge(edits[['segment', 'user_name', 'user_affiliation']].drop_duplicates(), on=['segment', 'user_name'])
# Creating a new DataFrame for segment counts
segment_counts_df = edits.groupby('segment').size().reset_index(name='total_edit_count')
# Merging with the segment_counts_df DataFrame
user_name_counts = user_name_counts.merge(segment_counts_df, on='segment')
# Merging with the segment_publication_df DataFrame to get 'symbol'
user_name_counts = user_name_counts.merge(segment_publication_df[['Updated_seg_ids', 'symbol']], left_on='segment', right_on='Updated_seg_ids', how='left')
# Drop the redundant 'Updated_seg_ids' column
user_name_counts = user_name_counts.drop('Updated_seg_ids', axis=1)
# Calculating the 'user_percent_edited'
user_name_counts['user_percent_edited'] = round(user_name_counts['user_edit_count'] / user_name_counts['total_edit_count'] * 100)
# Calculating the percentage sum for each 'user_affiliation' group within the same 'segment'
user_affiliation_percentage_sum = user_name_counts.groupby(['segment', 'user_affiliation'])['user_percent_edited'].sum().reset_index(name='lab_percent_edited')
# Merging back into the user_name_counts DataFrame
user_name_counts = user_name_counts.merge(user_affiliation_percentage_sum, on=['segment', 'user_affiliation'])
# Sort the new DataFrame by 'user_affiliation'
user_name_counts = user_name_counts.sort_values(by='user_affiliation')


# Summing user_edit_count for each user_name
user_name_counts = user_name_counts[user_name_counts['lab_percent_edited'] >= 10].copy()
unique_users_df = user_name_counts.groupby('user_name', as_index=False).agg({
    'user_affiliation': 'first',  # Assuming user_affiliation is the same for all rows of a user
    'user_edit_count': 'sum'
})

# Sort the new DataFrame by 'user_affiliation'
unique_users_df = unique_users_df.sort_values(by='user_affiliation')

# Reordering columns
unique_users_final = unique_users_df[['user_name', 'user_affiliation', 'user_edit_count']].copy()

In [33]:
unique_users_final

,user_name,user_affiliation,user_edit_count
31,Eleni Samara,,3
96,Wes Murfin,,2
5,Amalia Braun,Alexander Borst Lab,34
2,Alisa Poh,Barry Dickson Lab,5
15,Beas Dutta,Bassem Hassan Lab,24
...,...,...,...
18,Binglin Huang,Sung Soo Kim Lab,1
29,Dustin Garner,Sung Soo Kim Lab,712
48,Jasper Phelps,Wei-Chung Lee lab,2
111,zequan liu,"Xueying ""Snow"" Wang (Lichtman Lab)",58


In [43]:
user_name_counts_final = user_name_counts[['segment', 'symbol', 'user_name', 'user_affiliation','user_edit_count','total_edit_count', 'lab_percent_edited']].copy()
user_name_counts_final = user_name_counts_final[user_name_counts_final['lab_percent_edited'] >= 10]
user_name_counts_final = user_name_counts_final.astype(str).copy()
user_name_counts_final['segment'] = user_name_counts_final['segment'].astype(str)
user_name_counts_final.head()

,segment,symbol,user_name,user_affiliation,user_edit_count,total_edit_count,lab_percent_edited
1236,720575940613374873,Tm9,Eleni Samara,,2,6,33.0
8181,720575940631934039,Tm9,Eleni Samara,,1,3,33.0
1561,720575940614774959,Tm16,Wes Murfin,,2,4,50.0
5936,720575940626393858,Tm9,Amalia Braun,Alexander Borst Lab,1,1,100.0
5670,720575940625753744,Tm9,Amalia Braun,Alexander Borst Lab,3,3,100.0


In [44]:
type(user_name_counts_final['segment'][1236])

str

In [45]:
#Saving in a new file

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
file_name = f'segment_ids_edit_counts_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
unique_users_final = unique_users_final.astype(str).copy()
unique_users_final.to_excel(savePath, sheet_name='User-Lab summary')


#More dataframes in same excel file
book = load_workbook(savePath)
writer = pd.ExcelWriter(savePath, engine = 'openpyxl')
writer.book = book
user_name_counts_finall = user_name_counts_final.astype(str).copy()
user_name_counts_final.to_excel(writer, sheet_name='Edits passed 10 percent')
writer.save()
writer.close()

In [46]:
savePath

'D:\\FlyWire-Data\\FlyWire-Authorship\\segment_ids_edit_counts_15Jan2024.xlsx'

### Current version to execute ???. NO.

In [6]:
# Step 1: Count the number of entries for each unique "segment"
segment_counts = edits['segment'].value_counts().reset_index()
segment_counts.columns = ['segment', 'total_segment_count']

# Step 2: Calculate the total number of entries for each "user_affiliation" within each unique "segment"
user_affiliation_counts = edits.groupby(['segment', 'user_affiliation'])['user_affiliation'].count().reset_index(name='user_affiliation_count')
user_affiliation_counts = user_affiliation_counts.merge(segment_counts, on='segment')
user_affiliation_counts['affiliation_percentage'] = round(user_affiliation_counts['user_affiliation_count'] / user_affiliation_counts['total_segment_count'] * 100)

# Step 3: Calculate the total number of entries for each "user_name" within each unique "segment"
user_name_counts = edits.groupby(['segment', 'user_name'])['user_name'].count().reset_index(name='user_name_count')
user_name_counts = user_name_counts.merge(segment_counts, on='segment')
user_name_counts['user_percentage'] = round(user_name_counts['user_name_count'] / user_name_counts['total_segment_count'] * 100)

In [7]:
user_name_counts.head()


,segment,user_name,user_name_count,total_segment_count,user_percentage
0,720575940602880736,Arti Yadav,1,1,100.0
1,720575940602943968,Dustin Garner,13,13,100.0
2,720575940603472864,Zairene Lenizo,4,4,100.0
3,720575940603557920,Dustin Garner,1,1,100.0
4,720575940603809452,Shaina Mae Monungolh,11,11,100.0


In [8]:
user_affiliation_counts.head()

,segment,user_affiliation,user_affiliation_count,total_segment_count,affiliation_percentage
0,720575940602880736,Greg Jefferis Lab,1,1,100.0
1,720575940602943968,Sung Soo Kim Lab,13,13,100.0
2,720575940603472864,"Mala Murthy Lab, Sebastian Seung Lab",4,4,100.0
3,720575940603557920,Sung Soo Kim Lab,1,1,100.0
4,720575940603809452,"Mala Murthy Lab, Sebastian Seung Lab",11,11,100.0


In [9]:
# Merge the two DataFrames on the 'segment' column
edit_counts = pd.merge(user_affiliation_counts[['segment', 'user_affiliation','user_affiliation_count', 'affiliation_percentage', 'total_segment_count']], user_name_counts[['segment', 'user_name', 'user_name_count','user_percentage']], on='segment')


# Sort the DataFrame by 'user_name'
edit_counts = edit_counts.sort_values(by='user_name')

# Reset the index to have a clean index order
edit_counts = edit_counts.reset_index(drop=True)

# Filtering the data set to >10% edits bb lab
edit_counts_final = edit_counts[['segment','user_name', 'user_name_count', 'user_affiliation','user_affiliation_count', 'total_segment_count','affiliation_percentage']].copy()
edit_counts_final = edit_counts_final[edit_counts_final['affiliation_percentage']>= 10].copy()

# Print the merged DataFrame
display(edit_counts_final)

,segment,user_name,user_name_count,user_affiliation,user_affiliation_count,total_segment_count,affiliation_percentage
1,720575940635089515,A. Javier,1,Eyewire,38,39,97.0
2,720575940625388365,A. Javier,2,Eyewire,2,12,17.0
3,720575940625388365,A. Javier,2,Greg Jefferis Lab,6,12,50.0
4,720575940625388365,A. Javier,2,"Mala Murthy Lab, Sebastian Seung Lab",4,12,33.0
5,720575940619717717,Albert Lin,1,Eyewire,4,5,80.0
...,...,...,...,...,...,...,...
19744,720575940638682458,st0ck53y,3,Eyewire,3,15,20.0
19745,720575940619426507,st0ck53y,2,Eyewire,2,2,100.0
19746,720575940638682458,st0ck53y,3,"Mala Murthy Lab, Sebastian Seung Lab",12,15,80.0
19748,720575940632427576,zequan liu,6,Eyewire,136,153,89.0


In [10]:
#Saving in a new file

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
file_name = f'segment_ids_edit_counts_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
edit_counts_final.to_excel(savePath, sheet_name='Edits passed 10 percent')


#More dataframes in same excel file
book = load_workbook(savePath)
writer = pd.ExcelWriter(savePath, engine = 'openpyxl')
writer.book = book
user_name_counts.to_excel(writer, sheet_name='Edits counts by user')
user_affiliation_counts.to_excel(writer, sheet_name='Edits counts by affiliation')
writer.save()
writer.close()

In [11]:
savePath

'D:\\FlyWire-Data\\database\\segment_ids_edit_counts_15Jan2024.xlsx'